# Indian Pines Dataset

IndianPines は，全21025区画($145\times 145$)のデータである．
全区画について220バンドの分光特性が計測されており，全区画のおよそ半分である10320区画について16カテゴリの土地被覆ラベルが当てられている．

10320区画における16カテゴリの土地被覆ラベルの内訳は以下の通りである．


In [ ]:
#(1)
#| echo: false
from IndianPines import load as load_pines

pines = load_pines(include_background=True,exclude_WaterAbsorptionChannels=False)

print(f'number of instances: {pines.features.shape[0]}')
print(f'dimension of feature vector (number of bands): {pines.features.shape[1]}')
print('='*10)
for i, t in enumerate(pines.target_names):
    n = pines.target[pines.target==i].shape[0]
    if t == 'background':
        print(f'{t}(non-studied): {n}')
    else:
        print(f'{t}: {n}')
print('='*10)

## 特徴空間について

220バンドのうち，水分吸収波帯である20バンドを除くデータを使うのが常である．
また，分光特徴は無相関ではないため，主成分分析を用いて，20次元に削減する．


In [ ]:
#(2)
#| echo: false
from IndianPines import recategorize17to10_csv

pines = load_pines(pca=20, include_background=True)

print(f'number of instances: {pines.features.shape[0]}')
print(f'dimension of feature vector (number of bands): {pines.features.shape[1]}')
print('='*10)
for i, t in enumerate(pines.target_names):
    n = pines.target[pines.target==i].shape[0]
    if t == 'BackGround':
        print(f'{t}(non-studied): {n}')
    else:
        print(f'{t}: {n}')
print('='*10)

## 土地被覆ラベルについて，

ラベルについては，いくつかの土地被覆は極端にサンプル数が少なかったり，同様の被覆をまとめることができ，もとの16ラベルから9ラベルにまとめることとする．

10320区画における9ラベルの内訳は以下の通りである．


In [ ]:
#(3)
#| echo: false
pines = load_pines(pca=20,recategorize_rule=recategorize17to10_csv, include_background=True)

print(f'number of instances: {pines.features.shape[0]}')
print(f'dimension of feature vector (number of bands): {pines.features.shape[1]}')
print('='*10)
for i, t in enumerate(pines.target_names):
    n = pines.target[pines.target==i].shape[0]
    if t == 'BackGround':
        print(f'{t}(non-studied): {n}')
    else:
        print(f'{t}: {n}')
print('='*10)

## トレーニングデータ，　テストデータ

以後，データを次のように呼称して区別する．

- 全区画(21025区画, 145\times 145)
  - アノテーションデータ(10320区画)
    - トレーニングデータ: 学習用
      - ラベル付きデータ: 教師付き学習用
      - ラベルなしデータ: 半教師付き学習用追加分
    - テストデータ: 分類性能評価用
  - バックグラウンドデータ(10705区画)

まず，アノテーションデータ 10320区画を，トレーニングデータとテストデータに区分けする．

1. アノテーションデータをカテゴリごとに，連続領域のクラスタに分ける
2. カテゴリごとに，広いクラスタの区画から順に，トレーニングデータを連続領域で50％確保し，残りをテストデータとする．双方等しく $N$ 区画である．
3. トレーニングデータからラベル付きデータを $N$ に対して $p_{l}$ の割合で連続的に確保する．
4. トレーニングデータからラベル付きデータを除いた区画から，$N$ に対して $p_{u}$ の割合でランダムに抽出する．


In [ ]:
#(4)
#| echo: false
from IndianPines import load
from s3vm_pines.module import recategorize17to10_csv, train_test_split3, labeled_unlabeled_sample, colored_map
from matplotlib import pyplot as plt
import numpy as np

pines = load(pca=20, include_background=True, recategorize_rule=recategorize17to10_csv)

train_test_status, train_test_status_name = train_test_split3()

n_train = len(train_test_status[train_test_status==2])
n_test = len(train_test_status[train_test_status==1])
p_train = n_train/(n_train+n_test)

status, status_name = labeled_unlabeled_sample(p_labeled = 0.5, p_unlabeled = 0.5, train_test_status = train_test_status)


print("target                  | labeled   | unlabeled | train-rest| test")
for t in range(len(pines.target_names)):
    print(f"{pines.target_names[t]:23s}\t",end='')
    for s in [3, 4, 2, 1]:
        print(f"| {pines.target[(status==s) & (pines.target==t)].shape[0]:9d} ",end='')
    print("\r")

fig1 = plt.figure(figsize=(6.4*3,4.8), constrained_layout=True)

ax11 = fig1.add_subplot(1,3,1)
colored_map(ax11, pines.target[status==3], pines.coordinates[status==3],with_legend=False)
ax11.set_title('labeled')

ax12 = fig1.add_subplot(1,3,2)
colored_map(ax12, pines.target[status==4], pines.coordinates[status==4],with_legend=False)
ax12.set_title('unlabeled')

ax13 = fig1.add_subplot(1,3,3)
colored_map(ax13, pines.target[status==1], pines.coordinates[status==1])
ax13.set_title('test')